## Langchain: Let's Start With The Basics

This is a Python notebook. It allows us to run code segments and see the results. There's a series of imports that we'll do first and then load the models from OpenAI

[LangChain Providers](https://python.langchain.com/docs/integrations/providers)

NOTE: You will need to have an environment variable of OPENAI_API_KEY with your key value

In [ ]:
pip install -q openai langchain

In [ ]:
from langchain.chat_models.openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo',
             temperature=1,
             max_tokens = 256)

In [ ]:
prompt = "How much wood would a wood chuck chuck?"
print(llm.invoke(prompt))

# Prompt Templates

Passing in text is limiting as it requires the context to be continued be passed in. Enter the first concepts of working with LLMs and that is the idea of a prompt template. We define these templates with variables that we can then pass in to fill in the template before it's passed to the LLM to complete.

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain


restaurant_template = """
I want you to act as a naming consultant for new restaurants.

Return a list of restaurant names. Each name should be short, catchy and easy to remember.
It should relate to the type of restaurant you are naming.

What are some good names for a restaurant that is {restaurant_description}?
"""

# An example prompt with one input variable
prompt_template = PromptTemplate(input_variables=["restaurant_description"], template=restaurant_template)

# we create our first chain! This uses the llm and also allows us to pass in a prompt template
chain = LLMChain(llm=llm,
                 prompt=prompt_template)


In [ ]:
description1 = "a chinese place that serves dim sum and other specialties"
description2 = "an american place that is themed with baseball memorabilia"
description3 = "a cafe that has live hard rock music and memorabilia"

# Run the chain only specifying the input variable.
print(chain.run(description1))

# Zero and Few Shot Learning

Notice that in the last example we didn't really prime the model with any examples of the output. This is called zero-shot prompting. We just sent in some information to the LLM and let it go. There's also a way for us to give it a few examples and then ask it to generate response. See this example below.

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate

# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

In [ ]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [ ]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [ ]:

# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="big"))

In [ ]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt)

# Run the chain only specifying the input variable.
print(chain.run("big"))